## ПОСТАНОВКА ЗАДАЧИ. УЧИМСЯ ЧИТАТЬ UTM-МЕТКИ.

<img src="https://blog.pruffme.com/wp-content/uploads/2017/09/utm_labels-1024x433.png" alt="кортинка">

В этом блоке модуля мы разберем применение сложных функций к датафреймам. Этот прием значительно расширит ваши возможности по обработке данных и позволит быстро решать огромное количество задач высокой сложности.

Мы будем рассматривать новый датасет по рекламным кампаниям в Яндекс.Директе. В нем у каждой кампании есть UTM-метка. 

[Файл с кампаниями Яндекс.Директа](http://study.skillfactory.ru/c4x/Skillfactory/PYB-1/asset/ad_campaigns.xlsx)

Наша задача посчитать какая кампания имеет максимальную сумму по столбцу "Продуктивность". Проблема в том, что название кампании "спрятано" внутри ссылки в столбце "Ссылка". Например, так:

https://awesome-site.ru/?utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}

Давайте посмотрим как можно извлечь utm_campaign и другие параметры ссылки.

Для тех, кто не знает, что такое UTM-метки - это параметры, которые добавляют в ссылку, ведущую на сайт. Их названия (utm_source, utm_medium итд) сложились исторически. Эти параметры нужны для того, чтобы получить подробную информацию о каждом источнике трафика. Например, utm_source=yandex внутри ссылки означает, что источник нашего трафика Яндекс, а  utm_campaign=a825749b87 означает номер рекламной кампании. Счетчики рекламных систем и аналитики умеют автоматически распознавать эти метки в ссылках и присваивать визиту корректный источник трафика. Для работы с датасетом такого понимания должно быть достаточно.

Подробно про метки для расширения кругозора можно почитать, например, [тут](http://tilda.education/articles-how-to-use-utm-url). 

## ПАРСИНГ ССЫЛОК
## ШАГ 1. ЧИТАЕМ ФАЙЛ

Раз уж наша задача началась с технических проблем со ссылками, то начнем с них. Исходный файл дан в формате Excel, поэтому для его чтения используем метод read_excel:

In [38]:
import pandas as pd
pd.set_option('display.max_rows', 10)

In [5]:
ad_stats = pd.read_excel('./module04_files/ad_campaigns.xlsx')
ad_stats.head()

,Название группы,Фраза (с минус-словами),Продуктивность,ID объявления,Заголовок,Текст,Ссылка
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...


## ШАГ 2. ПЕРЕИМЕНУЕМ СТОЛБЦЫ

Заголовки столбцов имеют кириллические названия. Хорошей практикой является их замена на латинские эквиваленты, т. к. это исключает возможные проблемы с кодировкой (не все операционные системы и программы корректно распознают кодировку русских и других символов).

Для переименования используем метод rename:

In [6]:
ad_stats.rename(columns={'Название группы': 'group', 'Фраза (с минус-словами)': 'phrase', 
                         'Продуктивность': 'effect', 'ID объявления': 'ad_id', 'Заголовок': 'title', 
                         'Текст': 'text', 'Ссылка': 'link'}, inplace=True)
ad_stats.head()

,group,phrase,effect,ad_id,title,text,link
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...


## БИБЛИОТЕКА URLLIB
Для решения задач по разбивке ссылок нам пригодится библиотека Urllib. Эта библиотека позволяет автоматически распознавать составные части ссылок: имя сайта, протокол, параметры итп.

Возьмем для примера ссылку из первой строки:

In [7]:
url = ad_stats.loc[0, 'link']

In [8]:
url

'https://awesome-site.ru/?utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}'

In [9]:
from urllib import parse

parsed = parse.urlsplit(url)
parsed

SplitResult(scheme='https', netloc='awesome-site.ru', path='/', query='utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}', fragment='')

## ПОЛУЧЕНИЕ ПАРАМЕТРОВ ССЫЛКИ
Что значит такая запись?

Теперь мы можем обращаться к объекту parsed и получать разобранные части ссылки. Например, чтобы получить название домена, пишем:

In [10]:
print(parsed.netloc)

awesome-site.ru


## ПРАКТИЧЕСКАЯ ЗАДАЧА:
Запишите параметры ссылки в переменную query_string, мы будем дальше с ней работать. Т. е. чтобы при выводе переменной query_string на экран получить 'utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}'.

In [11]:
query_string = parsed.query

## ИЗВЛЕКАЕМ НАЗВАНИЯ КАМПАНИЙ
Итак, имея параметры ссылки в переменной query_string, можно достать параметр utm_campaign. Для этого проще всего использовать метод parse_qs. Он позволяет автоматически преобразовывать строку с параметрами ссылки query_string из предыдущего шага с словарь. Посмотрим как он работает: 

In [12]:
params_dict = parse.parse_qs(query_string)

params_dict

{'utm_source': ['yandex'],
 'utm_medium': ['cpc'],
 'utm_campaign': ['a825749b87'],
 'utm_content': ['dev_{device_type}']}

Метод возвращает значения каждого параметра внутри списка (на случай, если один из параметров будет содержаться в ссылке несколько раз). В итоге название кампании получаем, просто обращаясь к словарю:

In [13]:
params_dict['utm_campaign'][0]

'a825749b87'

## КАМПАНИИ ИЗ ДАТАФРЕЙМА


Мы научились доставать информацию из одной ссылки, но у нас могут быть миллионы таких ссылок в датасете! Что же делать? 

На помощь придет функция! 

## ШАГ 1. НАПИШЕМ ФУНКЦИЮ

Давайте запишем аналогичные действия в виде функции, которую можно применять к датафрейму.

На вход будет подаваться каждая строка датафрейма, которую можно обозначить как row (как обычно название переменной можно выбирать любым удобным вам). С помощью такого способа вы можете сделать функцию обработки строк любой сложности. В тройных кавычках обычно пишут описание что делает функция.

In [14]:
def campaign_name(row):
    """Получение названия кампании из ссылки внутри строки row"""

    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)

    return params_dict['utm_campaign'][0]

## ШАГ 2. ПРИМЕНИМ ФУНКЦИЮ

Чтобы применить функцию к датафрейму, используйте метод apply. Параметр axis указывает на то, что в функцию в параметр row должна быть передана строка (в случае axis=0 будет передан столбец):

In [15]:
ad_stats['campaign'] = ad_stats.apply(campaign_name, axis=1)

ad_stats.head()

,group,phrase,effect,ad_id,title,text,link,campaign
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca


## Упражнение
(1 балл из 1)
Для каждого значения столбца campaign посчитайте количество символов в названии кампании (точнее это хэш от названия кампании).

Какое в среднем количество символов в названии кампаний? Ответ округлите до ближайшего целого числа

In [17]:
ad_stats['campaign'].apply(lambda x: len(x)).mean()

10.0

## ПОСТАНОВКА ЗАДАЧИ
В этом блоке мы проанализируем показатели рекламных кампаний, сгруппированных по различным столбцам. В прошлых модулях вы уже сталкивались с таким действием, когда использовали метод value_counts. Он возвращает количество строк по заданному столбцу. Теперь мы хотим расширить эту задачу на более общий случай - необходимо  для заданного столбца или нескольких столбцов посчитать определенные метрики. Например, найти кампании с максимальной и минимальной суммой столбца effect.

Для таких и более сложных задач предназначен метод groupby, который является аналогом GROUP BY в SQL-командах. В этом блоке мы научимся группировать датафреймы по одному и нескольким столбцам, а также  получать несколько метрик одной командой.

## ГРУППИРОВКА ПО ОДНОМУ СТОЛБЦУ
Будем использовать датафрейм ad_stats с прошлого шага со столбцом названий кампаний  'campaign'. 

Вспомним как мы считали количество строк для каждой кампании:

In [18]:
ad_stats['campaign'].value_counts().head()

7f35591a28    456
8e77a0e565    348
2cc2e7d770    240
f6d2ae1e3d    220
e90f4db55a    218
Name: campaign, dtype: int64

Аналогичный результат можно получить более универсальным методом groupby.

Например, укажем столбец, по которому мы группируем ('campaign') и метрику, которую хотим посчитать для каждого столбца датафрейма (количество строк count):

In [21]:
ad_stats.groupby('campaign').count().head()

,group,phrase,effect,ad_id,title,text,link
campaign,,,,,,,
034ada41b6,3,3,3,3,3,3,3
03fcaecd1d,2,2,2,2,2,2,2
0697a81555,2,2,2,2,2,2,2
08cdcb57a3,3,3,3,3,3,3,3
0f0ba311fb,2,2,2,2,2,2,2


## ИНДЕКСЫ ДАТАФРЕЙМОВ
В прошлом шаге столбец 'campaign', по которому мы группировали датафрейм, оказался смещенным. Это означает, что он стал индексом датафрейма. Т. е. вместо номеров строк теперь у нас названия кампаний. Чтобы перевести этот результат в привычный датафрейм добавим команду reset_index:

In [22]:
ad_stats.groupby('campaign').count().reset_index().head()

,campaign,group,phrase,effect,ad_id,title,text,link
0,034ada41b6,3,3,3,3,3,3,3
1,03fcaecd1d,2,2,2,2,2,2,2
2,0697a81555,2,2,2,2,2,2,2
3,08cdcb57a3,3,3,3,3,3,3,3
4,0f0ba311fb,2,2,2,2,2,2,2


Шаг пересчета индекса не является чем-то обязательным. Но может избавить от трудностей при дальнейшей работе с полученным датафреймом. Теперь с помощью groupby мы можем использовать гораздо больше функций для группировки данных, чем мы имели с методом value_counts.

## ГРУППИРОВКА С СУММОЙ
Добавим сортировку, чтобы получить результат, аналогичный методу value_counts:

In [24]:
(ad_stats.groupby('campaign')
         .count()
         .reset_index()
         .sort_values('group', ascending=False)
         .head())

,campaign,group,phrase,effect,ad_id,title,text,link
62,7f35591a28,456,456,456,456,456,456,456
69,8e77a0e565,348,348,348,348,348,348,348
22,2cc2e7d770,240,240,240,240,240,240,240
110,f6d2ae1e3d,220,220,220,220,220,220,220
105,e90f4db55a,218,218,218,218,218,218,218


В исходной задаче нам нужно посчитать сумму столбца effect для каждой кампании. Для этого заменим метод count на сумму:

In [31]:
ad_stats.head()

,group,phrase,effect,ad_id,title,text,link,campaign
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca


In [30]:
(ad_stats.groupby('campaign')
         .sum()
         .sort_values('effect')
         .head()
)

,effect,ad_id
campaign,,
63153306ff,2.8,2428554756
f80e902d76,3.3,2101705003
5a7508fa8d,3.5,2428554764
d5b16c7769,3.6,3401169381
7b17b2d188,5.8,2101705006


## Упражнение
(1 возможный балл)
Для каждой кампании campaign найдите минимальное значение столбца effect и постройте рейтинг кампаний по этим значениям, начиная с самого низкого.

Какая кампаний будет на второй строке этого рейтинга? Пример формата ответа: d96029561c

In [39]:
(ad_stats.groupby('campaign')
         .agg({'effect': min})
         .sort_values(by='effect')
)

,effect
campaign,
8e77a0e565,1.6
7f35591a28,1.7
be97523396,1.8
d96029561c,1.8
acc3077e55,1.9
...,...
9ee3e2c876,7.7
28d8dca4df,8.4
eb577faed1,8.5


## ПОЛУЧЕНИЕ НЕСКОЛЬКИХ МЕТРИК
Мы уже почти решили первоначальную задачу. Давайте посмотрим еще несколько возможностей метода groupby. Для получения нескольких метрик просто укажите их списком в методе agg.
Чтобы итоговая таблица не была слишком большой укажем сразу, что нам нужен результат для столбца 'effect':

In [41]:
ad_stats.groupby('campaign').agg(['min', 'max'])['effect'].head()

,min,max
campaign,,
034ada41b6,4.1,9.0
03fcaecd1d,4.0,6.5
0697a81555,4.0,5.5
08cdcb57a3,5.5,7.4
0f0ba311fb,4.4,6.4


## РАЗНЫЕ МЕТРИКИ ДЛЯ РАЗНЫХ СТОЛБЦОВ
Наконец, можно сразу получить разные метрики для разных столбцов. Давайте проверим рекламные фразы какой длины имеют большее значение столбца effect. Для этого добавим столбец 'phrase_length' с длиной поисковой фразы:

In [42]:
ad_stats['phrase_length'] = ad_stats.apply(lambda row: len(row['phrase']), axis=1)

ad_stats.head()

,group,phrase,effect,ad_id,title,text,link,campaign,phrase_length
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,21
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,19
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,54
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca,11
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca,9


И для каждой кампании для столбца 'effect' посчитаем минимальное и максимальное значение, а для столбца 'phrase_length' - среднее значение. Тем самым мы получим логичные метрики для каждого столбца в отдельности:

In [44]:
(ad_stats.groupby('campaign')
         .agg({'effect': ['min', 'max'], 
               'phrase_length': 'mean'})
         .head())

effect      phrase_length
              min  max          mean
campaign                            
034ada41b6    4.1  9.0     38.333333
03fcaecd1d    4.0  6.5     22.000000
0697a81555    4.0  5.5     27.000000
08cdcb57a3    5.5  7.4     24.000000
0f0ba311fb    4.4  6.4     27.000000

## ГРУППИРОВКА ПО НЕСКОЛЬКИМ СТОЛБЦАМ
Группировать датафреймы можно по нескольким столбцам одновременно. Давайте посмотрим сколько и каких различных рекламных объявлений входит в каждую группу (столбец 'group').
Добавим сортировку, чтобы сразу увидеть максимальное число объявлений в одной группе

In [49]:
(ad_stats.groupby(['group', 'campaign'])
         .count()
         .sort_values('phrase', ascending=False)
#          .reset_index()
         .head(20))

,,phrase,effect,ad_id,title,text,link,phrase_length
group,campaign,,,,,,,
мрт на алексеевской кулаков переулок 13,7f35591a28,3,3,3,3,3,3,3
мрт каширское шоссе,7f35591a28,3,3,3,3,3,3,3
мрт кутузовский,7f35591a28,3,3,3,3,3,3,3
мрт куркинское шоссе 30,7f35591a28,3,3,3,3,3,3,3
мрт куркино,7f35591a28,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...
мрт калужское шоссе 9,7f35591a28,3,3,3,3,3,3,3
мрт иваньковское,7f35591a28,3,3,3,3,3,3,3
мрт жулебино,7f35591a28,3,3,3,3,3,3,3


В нашем датафрейме у каждой кампании не более трех объявлений. Обратите внимание, что теперь индекс датафрейма состоит сразу из двух столбцов. Вернуть датафрейм в привычный вид можно с помощью метода  reset_index.

## Проверочное задание
(1 балл из 1)
Итак, давайте ответим на вопрос блока.

Какая кампания имеет наименьшую сумму по столбцу effect? Пример ответа: 7f35591a28

In [51]:
ad_stats.groupby('campaign').agg(sum).sort_values(by='effect')

,effect,ad_id,phrase_length
campaign,,,
63153306ff,2.8,2428554756,27
f80e902d76,3.3,2101705003,18
5a7508fa8d,3.5,2428554764,22
d5b16c7769,3.6,3401169381,19
7b17b2d188,5.8,2101705006,14
...,...,...,...
e90f4db55a,842.0,458171988130,6182
7388fb4c7b,896.9,437154834440,4288
2cc2e7d770,966.8,512572671002,6080


# Домашнее задание

## ПОСТАНОВКА ЗАДАЧИ
Работаем с файлом keywords.csv из первой части модуля.

Наша задача - написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения зададим довольно простыми:

In [52]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Регионов, по которым будем классифицировать запросы, будет три: Центр, Северо-Запад и Дальний Восток. Вам необходимо написать функцию geo_classification, внутри которой происходит следующий алгоритм:

1. Для каждой строки датафрейма row пробегаем ключи и значения словаря geo_data. Напомним, для этого синтаксис в питоне такой: 
```python
for region, city_list in geo_data.items():
    ```
2. Для каждого города из списка city_list необходимо проверить содержится ли его название в поисковом запросе строки row (т. е. в row['keyword']). Если содержится, то возвращаем значение region.

3. Если после всех циклов ни один город не подошел, то возвращаем значение 'undefined'.

Замечание. Реализовать классификацию по регионам можно множеством способов. Здесь мы выбрали один из них, чтобы проверить как вы научились применять материалы модуля. Пожалуйста, следуйте указанному алгоритму, чтобы ваш ответ сошелся с онлайн-проверкой. В реальных задачах вы легко сможете написать гораздо более совершенные классификаторы.

In [53]:
hw_data = pd.read_csv('./module04_files/keywords.csv')
hw_data.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [91]:
def geo_cat(row):
#     print(row)
    for region, cities in geo_data.items():
        for c in cities:
            if c in row[0]:
                return region
    return 'undefined'

In [93]:
hw_data['geo_cat'] = hw_data.apply(lambda row: geo_cat(row), axis=1)

In [95]:
hw_data.groupby('geo_cat').count()

,keyword,shows
geo_cat,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387
